In [ ]:

# 問題なく動いている。このセルは編集しない。
# 指定したフォルダについて格納されたファイルを動画か、画像か判定し、ファイルタイプに応じた生成日時取得の関数に飛んで生成日時を取得する。
# 得られた生成日時を用いて、ファイル名先頭に生成日時を追加したファイル名で複製され、元のフォルダの下のoutputというフォルダに格納される。


import os
import time
import shutil

def copy_with_new_name(file_path, prefix, destination_folder):
    # # ファイルの作成日時を取得
    # creation_time = os.path.getctime(file_path)
    # creation_datetime = time.strftime("%Y-%m-%d-%H-%M", time.gmtime(creation_time))
    
    # 指定されたプレフィックスを付け加えた新しいファイル名を作成
    base, ext = os.path.splitext(os.path.basename(file_path))
    new_file_name = f"{prefix}_{base}{ext}"
    new_file_path = os.path.join(destination_folder, new_file_name)
    
    # 同じ名前のファイルが既に存在する場合、サフィックスを追加（例：(1)）
    count = 1
    while os.path.exists(new_file_path):
        new_file_name = f"{prefix}_{base} ({count}){ext}"
        new_file_path = os.path.join(destination_folder, new_file_name)
        count += 1
    
    # ファイルを指定されたフォルダにコピー
    shutil.copy(file_path, new_file_path)
    print(f"{os.path.basename(file_path)} を {new_file_path} にコピーしました")

# # 例として使用するファイルを指定
# file_to_copy = 'C:\\Users\\taqsa\\Documents\\Temp 動画編集用\\伊那谷\\test\\PRO_VID_20240406_173430_240.mp4'  # ファイルのパスを指定
# new_file_prefix = '2024-04-06-17-34'  # 新しいファイル名の先頭に付け加えるプレフィックスを指定
# destination_folder = 'C:\\Users\\taqsa\\Documents\\Temp 動画編集用\\伊那谷\\output'  # コピー先のフォルダを指定
# copy_with_new_name(file_to_copy, new_file_prefix, destination_folder)

import os
import time
import shutil
import mimetypes
import ffmpeg
from PIL import Image
import datetime
from datetime import datetime, timedelta


def process_file(file_path):
    # ファイルのMIMEタイプを取得
    mime_type, _ = mimetypes.guess_type(file_path)

    if mime_type:
        if mime_type.startswith('video'):
            # 動画の場合は関数Aを呼び出す
            prefix = process_video(file_path)
            copy_with_new_name(file_path, prefix, destination_folder)
                

        elif mime_type.startswith('image'):
            # 画像の場合は関数Bを呼び出す
            prefix = process_image(file_path)
            copy_with_new_name(file_path, prefix, destination_folder)
            
        else:
            print(f"未知のファイルタイプ: {mime_type}")
    else:
        print(f"ファイルのMIMEタイプが判定できませんでした: {file_path}")


def process_video(video_path):
    # 動画処理のコードをここに記述
    print(f"動画を処理します: {video_path}")
    try:
        probe = ffmpeg.probe(video_path)
        creation_time = next((stream['tags']['creation_time'] for stream in probe['streams'] if 'creation_time' in stream['tags']), None)
        if creation_time:
            # UTCから日本時間に変換
            try:
            # creation_time の形式を確認して、必要に応じて format パラメータを変更する
                utc_time = datetime.strptime(creation_time, "%Y-%m-%dT%H:%M:%S.%fZ")
            except ValueError:
                print(f"creation_time の形式が正しくありません: {creation_time}")
                return
            japan_time = utc_time + timedelta(hours=9)
            print(japan_time)
            
            # フォーマットを変更
            formatted_time = japan_time.strftime("%Y-%m-%d-%H-%M")
            print(f"動画の撮影日時 (日本時間): {formatted_time}")
            return formatted_time
        else:
            print("撮影日時が見つかりませんでした。")
    except Exception as e:
        print(f"Error: {e}")
    
def process_image(image_path):
    from PIL import Image
    import datetime
    # 画像処理のコードをここに記述
    print(f"画像を処理します: {image_path}")
    try:
        with Image.open(image_path) as img:
            exif = img._getexif()
            # print(exif)
            formatted_datetime = None
            if not exif:
                raise Exception(f"画像 {image_path} にはExifデータがありません。")
            
            creation_datetime = exif.get(36867)
            if creation_datetime is None:
                print("作成日時が見つかりませんでした。")
                # または、適切な処理を追加
            else:
                formatted_datetime = datetime.datetime.strptime(creation_datetime, "%Y:%m:%d %H:%M:%S").strftime("%Y-%m-%d-%H-%M")
                print(f"フォーマットされた日時: {formatted_datetime}")
                return formatted_datetime
    except (KeyError, FileNotFoundError):
        raise Exception(f"画像 {image_path} の作成日時を抽出できませんでした。")
    
# フォルダ内のファイルを処理
folder_path = input("処理するファイルが格納されたフォルダのパス: ")    #"C:\\Users\\taqsa\\Documents\\Temp 動画編集用\\伊那谷\\test\\"
destination_folder = folder_path + "\\output"
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        process_file(file_path)

